<a href="https://colab.research.google.com/github/yilin2718/search-product-reviews-weaviate/blob/main/cap_proj_weaviate_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Weaviate Python client

https://weaviate-python-client.readthedocs.io/en/latest/index.html




In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
cd /content/drive/MyDrive/MLops3/


/content/drive/MyDrive/MLops3


In [3]:
pwd

'/content/drive/MyDrive/MLops3'

In [17]:
import sys
!{sys.executable} -m pip install weaviate-client==2.5.0 

     |████████████████████████████████| 56 kB 2.3 MB/s 
  Created wheel for validators: filename=validators-0.19.0-py3-none-any.whl size=19553 sha256=6506f712402a1aa386a5696983f2e966ae47d91188de0be2eff0b4c4828bc46b
  Stored in directory: /root/.cache/pip/wheels/fe/5d/69/ff53a908b9f14fb7730a58fdede0fac4cdc99ef3624ec76d05
Successfully built validators


In [18]:
#from getpass import getpass # hide password
import weaviate # to communicate to the Weaviate instance
from weaviate.tools import WCS

In [ ]:
my_credentials = weaviate.auth.AuthClientPassword(username=input("User name: "), password=input('Password: '))

In [21]:
my_wcs = WCS(my_credentials)

***Create a cluster***

In [22]:
cluster_name = 'sridiyer-first-weaviate-instance'
weaviate_url = my_wcs.create(cluster_name=cluster_name)
weaviate_url

100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


'https://sridiyer-first-weaviate-instance.semi.network'

***Connect to the cluster***


In [23]:

client = weaviate.Client(weaviate_url)

In [24]:
client.is_ready()

True

***Get Data***

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/Electronics.json.gz

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Electronics.json.gz

In [6]:
# get amazon product data

import pandas as pd
import gzip
import json

def parse(path):
  #g = gzip.open(path, 'rb')
  g = open(path, 'r')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
    if i == 10:
      break
  return pd.DataFrame.from_dict(df, orient='index')

data_fn = 'reviews_Electronics_5.json' # expected in gz format
df = getDF(data_fn)

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [ ]:
### load the meta data

gzFile_name = 'meta_Electronics.json.gz'
#gzFile_name = 'Electronics.json.gz'

data = []
with gzip.open('meta_Computers.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

df = pd.DataFrame.from_dict(data)


In [ ]:
# clean up loaded data
df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
print(len(df4))
print(len(df5))

***Create Schema for Weaviate***


In [39]:
product_class_schema = {
    # name of the class
    "class": "Product",
    "description" : "A Product class to store the product information and its reviewers",
    "properties" : [
                 {
                    "name": "asin",
                    "dataType": ["string"],
                    "description": "product id from amazon",
                },
                {
                    "name": "title",
                    "dataType": ["text"],
                    "description": "The title of the product",
                },
                {
                    "name": "description",
                    "dataType": ["text"],
                    "description": "The product description",
                },
                {
                    "name": "price",
                    "description": "The price of the wine in dollars",
                    "dataType": ["number"]
                },
                {
                    "name": "hasReviews",
                    "dataType": ["Review"],
                    "description": "List of reviews this product listing has ",
                }                   
    ]
}

In [40]:
review_class_schema = {
    "class": "Review",
    "description": "An Review class to store the review information",
    "properties": [
                {
                    "name": "overall",
                    "dataType": ["number"],
                    "description": "overall score from user ",
                },
                {
                    "name": "reviewText",
                    "dataType": ["text"],
                    "description": "review text",
                },
                {
                    "name": "summary",
                    "dataType": ["text"],
                    "description": "reviewer provided summary of review",
                },
                {
                    "name": "ofProduct",
                    "dataType": ["Product"],
                    "description": "The product of the review",
                } 
    ]
}

In [41]:
client.schema.create_class(product_class_schema)

UnexpectedStatusCodeException: ignored

In [42]:
# helper function
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))

prettify(client.schema.get())

{
  "classes": [
    {
      "class": "Product",
      "description": "A Product class to store the product information and its reviewers",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "properties": [
        {
          "dataType": [
            "string"
          ],
          "description": "product id from amazon",
          "name": "asin",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The title of the product",
          "name": "title",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The product description",
          "name": "description",
          "tokenization": "word"
        },
 

In [43]:
client.schema.create_class(review_class_schema)
prettify(client.schema.get())

{
  "classes": [
    {
      "class": "Product",
      "description": "A Product class to store the product information and its reviewers",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "properties": [
        {
          "dataType": [
            "string"
          ],
          "description": "product id from amazon",
          "name": "asin",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The title of the product",
          "name": "title",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The product description",
          "name": "description",
          "tokenization": "word"
        },
 

In [44]:
client.schema.property.create('Product', product_class_schema['properties'][4])


In [45]:
prettify(client.schema.get())

{
  "classes": [
    {
      "class": "Product",
      "description": "A Product class to store the product information and its reviewers",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "properties": [
        {
          "dataType": [
            "string"
          ],
          "description": "product id from amazon",
          "name": "asin",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The title of the product",
          "name": "title",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The product description",
          "name": "description",
          "tokenization": "word"
        },
 

In [37]:
schema = client.schema.get() # save schema
client.schema.delete_all() # delete all classes
prettify(client.schema.get())

{
  "classes": []
}


In [ ]:
client.schema.create(schema)
prettify(client.schema.get())